## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [13]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        # 初始化两层权重参数 (MNIST输入28x28=784)
        self.W1 = tf.Variable(np.random.randn(784, 256) * 0.01, dtype=tf.float32)
        self.b1 = tf.Variable(np.zeros(256), dtype=tf.float32)  # 添加第一层的偏置
        
        self.W2 = tf.Variable(np.random.randn(256, 10) * 0.01, dtype=tf.float32)
        self.b2 = tf.Variable(np.zeros(10), dtype=tf.float32)  # 添加第二层的偏置
        ####################


    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
         # 展平输入 (N, 28, 28) -> (N, 784)
        x = tf.reshape(x, [-1, 784])
        # 第一层: 输入层 -> 隐藏层 (带ReLU)
        x = tf.matmul(x, self.W1) + self.b1
        x = tf.maximum(0.0,x)
        # 第二层: 隐藏层 -> 输出层 (无激活函数)
        logits = tf.matmul(x, self.W2) + self.b2
        ####################
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [14]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [15]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.3020937 ; accuracy 0.1048
epoch 1 : loss 2.3018756 ; accuracy 0.10978334
epoch 2 : loss 2.3016572 ; accuracy 0.114483334
epoch 3 : loss 2.3014388 ; accuracy 0.11883333
epoch 4 : loss 2.3012204 ; accuracy 0.124466665
epoch 5 : loss 2.301002 ; accuracy 0.12913333
epoch 6 : loss 2.3007839 ; accuracy 0.13373333
epoch 7 : loss 2.3005655 ; accuracy 0.13895
epoch 8 : loss 2.3003469 ; accuracy 0.14386667
epoch 9 : loss 2.3001282 ; accuracy 0.14983334
epoch 10 : loss 2.2999094 ; accuracy 0.15563333
epoch 11 : loss 2.2996907 ; accuracy 0.16228333
epoch 12 : loss 2.2994716 ; accuracy 0.16821666
epoch 13 : loss 2.299252 ; accuracy 0.17465
epoch 14 : loss 2.299033 ; accuracy 0.1809
epoch 15 : loss 2.298813 ; accuracy 0.18761666
epoch 16 : loss 2.298593 ; accuracy 0.19458333
epoch 17 : loss 2.298373 ; accuracy 0.20296666
epoch 18 : loss 2.2981524 ; accuracy 0.20985
epoch 19 : loss 2.2979314 ; accuracy 0.21716666
epoch 20 : loss 2.2977104 ; accuracy 0.22416666
epoch 21 : loss 2.29748